In [48]:
import random
import math

In [49]:
#graf je predstavljen kao lista listi
#indeks liste predstavlja cvor
#element liste je lista cvorova sa kojima je dati cvor povezan
#ako se u grafu ne nalazi cvor u listi na indeksu tog cvora zapisujemo [-1]
class Graph:
    #ajdacency_list - lista listi
    #cost - niz cena cvorova
    #V - broj cvorova
    def __init__(self, adjacency_list, cost):
        self.cost = cost
        self.adj = adjacency_list
        self.V = len(self.adj)
    
    #DFS obilazak grafa
    #povratna vrednost - lista cvorova
    def DFSUtil(self, temp, v, visited):
        visited[v] = True
        temp.append(v)
        for i in self.adj[v]:
            if visited[i] == False:
                temp = self.DFSUtil(temp, i, visited)
        return temp
    
    #povratna vreednost - lista komponenti
    def connectedComponents(self):
        visited = []
        cc = []
        for i in range(self.V):
            #self.adj[i] != [] znaci da iz cvora polaze grane
            #self.adj[i][0] == -1 znaci da se cvor ne nalazi u grafu
            #posto se cvor ne nalazi u grafu zapisujemo kao da je posecen
            #time ne remetimo obilazak grafa
            if self.adj[i] != [] and self.adj[i][0] == -1:
                visited.append(True)
            else:
                visited.append(False)
        for v in range(self.V):
            if visited[v] == False:
                temp = []
                cc.append(self.DFSUtil(temp, v, visited))
        return cc
    #funkcija vraca novi graf koji ne sadrzi cvor node (kao ni njegove grane)
    #izbacujemo cvor i sve grane koje polaze iz tog cvora
    #zapisujemo -1 umesto liste grana koje polaze iz cvora
    #iz svih lista grana izbacujemo granu koja ide ka izbacenom cvoru
    def remove_node(self,node):
        newAdj = self.adj.copy()
        newAdj[node] = [-1]
        newAdj = [[ele for ele in sub if ele != node] for sub in newAdj]
        return Graph(newAdj, self.cost)

    #povratna vrednost funkcije je najveca cena cvora iz datog grafa
    def maxCost(self):
        maks = -float('inf')
        #prolazimo kroz sve cvovore koji se nalaze u grafu
        for i in range(self.V):
            #uslov u zagradi proverava da li je cvor u grafu
            if (self.adj[i] != [] and self.adj[i][0] != -1) or self.adj[i] ==[]:
                if self.cost[i] > maks:
                    maks = self.cost[i]
        return maks

In [50]:
#Provera klase Graph
adjacency_list = [[1,2,5], [0,5,2],[0,1,3,5], [2,4], [3,5],[0,1,2,4]]
vertics_cost = [8,4,1,6,5,4]
graph = Graph(adjacency_list, vertics_cost)
print(graph.adj)
print(graph.connectedComponents())
print(graph.maxCost())
graphWithout0 = graph.remove_node(0)
print(graphWithout0.adj)
print(graphWithout0.connectedComponents())
print(graphWithout0.maxCost())

[[1, 2, 5], [0, 5, 2], [0, 1, 3, 5], [2, 4], [3, 5], [0, 1, 2, 4]]
[[0, 1, 5, 2, 3, 4]]
8
[[-1], [5, 2], [1, 3, 5], [2, 4], [3, 5], [1, 2, 4]]
[[1, 5, 2, 3, 4]]
6


In [66]:
#klasa jedinki
#prosledjujemo joj pocetni graf
#jedinka je reprezentovana u obliku niza nula i jedinica (niz je velicine broja cvorova u grafu)
#nula oznacava da se cvor (indeks niza oznacava cvor) nalazi u jednom podgrafu
#a jedinica da se naalzi u drugom
class Individual:
    def __init__(self, graph):
        self.graph = graph
        self.code = []
        self.V = len(graph.adj)
        for i in range(self.V):
            self.code.append(random.random() < 0.5)
        self.fitness = self.fitnessFunction()
    #definise nacin poredjenja jedinki
    def __lt__(self, other):
        return self.fitness < other.fitness
    #racunamo u odnosu na broj komponenti podgrafova
    def penalty_function(self):
        graph1, graph2 = self.getSubgraphs()
        self.subgraph1 = graph1
        self.subgraph2 = graph2
        ncc1 = len(graph1.connectedComponents())
        ncc2 = len(graph2.connectedComponents())
                        
        max1 = self.subgraph1.maxCost()
        max2 = self.subgraph2.maxCost()
      
        return (ncc1-1)*max1 + (ncc2-1)*max2

    #fitnes raacuna koliko su priblizne vrednosti suma dva podgrafa
    #sto je fitnes manji resenje je bolje
    #penalti funcija sluzi kao kazneni poeni ako dobijemo podgrafove
    #koji nisu povezani
    def fitnessFunction(self):
        w1 = 0
        w2 = 0
        for i in range(self.V):
            if self.code[i]:
                w1 += self.graph.cost[i]
            else:
                w2 += self.graph.cost[i]

        return abs(w1-w2) + self.penalty_function()
    
    def getSubgraphs(self):
        graph1 = Graph(self.graph.adj.copy(), self.graph.cost)
        graph2 = Graph(self.graph.adj.copy(), self.graph.cost)
        for i in range(self.V):
            if self.code[i]:
                graph2 = graph2.remove_node(i)
            else:
                graph1 = graph1.remove_node(i)
        return graph1, graph2


In [67]:
#provera klase Individual
ind = Individual(graph)
print(ind.graph.adj)
print(ind.subgraph1.adj)
print(ind.subgraph2.adj)
print(ind.code)
print(ind.fitness)


[[1, 2, 5], [0, 5, 2], [0, 1, 3, 5], [2, 4], [3, 5], [0, 1, 2, 4]]
[[2], [-1], [0, 3], [2, 4], [3], [-1]]
[[-1], [5], [-1], [-1], [-1], [1]]
[True, False, True, True, True, False]
12


In [68]:
#turnir selekcija
# nTournament - velicina turnira
def selection(population):
    min = float('inf')
    for i in range(nTournament):
        j = random.randrange(len(population))
        if population[j].fitness < min:
            min = population[j].fitness
            k = j
    return k

In [69]:
def crossover(parent1, parent2, child1, child2):
    i = random.randrange(len(parent1.code))
    for j in range(i):
        child1.code[j] = parent1.code[j]
        child2.code[j] = parent2.code[j]
    for j in range(i, len(parent1.code)):
        child1.code[j] = parent2.code[j]
        child2.code[j] = parent1.code[j]

In [70]:
#mutacija
#probability - verovatnoca mutacije na genu
def mutation(individual):
    for i in range(len(individual.code)):
        if random.random() > probability:
            continue
        individual.code[i] = not individual.code[i]

In [71]:
#parametri genetskog algoritma
nTournament = 6
probability = 0.05
nPopulation = 100
nIterations = 500
nElite = 30 

In [77]:
#genetski algoritam
def genetic_algorithm(nPopulation, nIterations, nElite, nTournament, probability, graph):
    population = []
    newPopulation = []
    for i in range(nPopulation):
        population.append(Individual(graph))
        newPopulation.append(Individual(graph))

    for iteration in range(nIterations):
        population.sort()
        #sacuvam elitne jedinke - prvih nElite jedinki ostaju u populaciji
        for i in range(nElite):
            newPopulation[i] = population[i]

        for i in range(nElite, nPopulation, 2):
            k1 = selection(population)
            k2 = selection(population)
            crossover(population[k1], population[k2],newPopulation[i],newPopulation[i+1])
            mutation(newPopulation[i])
            mutation(newPopulation[i+1])
            newPopulation[i].fitness = newPopulation[i].fitnessFunction()
            newPopulation[i+1].fitness = newPopulation[i+1].fitnessFunction()

        population = newPopulation

    population.sort()
    return population[0]

In [78]:
def print_solution(solution):
    print('Solution: ')
    print('Graph1 - ', solution.subgraph1.adj)
    print('Graph2 - ', solution.subgraph2.adj)
    print('Fitness - ', solution.fitness)
    print('Code - ', solution.code)

In [82]:
adjacency_list = [[1,2,5], [0,5,2],[0,1,3,5], [2,4], [3,5],[0,1,2,4]]
vertics_cost = [8,4,1,6,5,4]
graph = Graph(adjacency_list, vertics_cost)
solution = genetic_algorithm(nPopulation, nIterations, nElite, nTournament, probability, graph)
print_solution(solution)


Solution: 
Graph1 -  [[-1], [5], [-1], [-1], [5], [1, 4]]
Graph2 -  [[2], [-1], [0, 3], [2], [-1], [-1]]
Fitness -  2
Code -  [False, True, False, False, True, True]


In [84]:
adjacency_list = [[1,2], [0,4],[0,3],[2,4], [1,3]]
vertics_cost = [3,7,1,4,9]
graph = Graph(adjacency_list, vertics_cost)
solution = genetic_algorithm(nPopulation, nIterations, nElite, nTournament, probability, graph)
print_solution(solution)

Solution: 
Graph1 -  [[-1], [-1], [-1], [4], [3]]
Graph2 -  [[1, 2], [0], [0], [-1], [-1]]
Fitness -  2
Code -  [False, False, False, True, True]


In [85]:
vertics_cost = [1,2,3,4,5,6,6,5,4,1,2,3,3,2,1,2,5,4,3,2,4,3,2,4,2,1,3,1,1,1,3,1]
adjacency_list = [[1,3],[0,3,20,11,6,28],[9,10,24],[0,1,9,12,4,5],[3,19,5],[3,4], [1,7,8], [6,8,14],[6,7,9], [8,20,3,2,10,12,30,23,16,29,15],[2,17,9],[1,13,18,25,12],[11,3,9],[18,11],[7,15,27],[14,12],[22,23,17,9],[16,10],[19,11,27,13],[18,27,4,31,26],[1,9],[28,22],[29,21,16],[16,9,24,30],[23,31,2],[28,11,29],[19,31],[14,18,19],[21,1,25],[25,22,9],[23,9],[19,26,24]]
graph = Graph(adjacency_list, vertics_cost)
solution = genetic_algorithm(nPopulation, nIterations, nElite, nTournament, probability, graph)
print_solution(solution)

Solution: 
Graph1 -  [[1, 3], [0, 3, 28], [-1], [0, 1, 4, 5], [3, 19, 5], [3, 4], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [27], [-1], [22, 23, 17], [16], [-1], [27, 4, 31, 26], [-1], [28, 22], [21, 16], [16], [-1], [-1], [19, 31], [14, 19], [21, 1], [-1], [-1], [19, 26]]
Graph2 -  [[-1], [-1], [9, 10, 24], [-1], [-1], [-1], [7, 8], [6, 8], [6, 7, 9], [8, 20, 2, 10, 12, 30, 29, 15], [2, 9], [13, 18, 25, 12], [11, 9], [18, 11], [-1], [12], [-1], [-1], [11, 13], [-1], [9], [-1], [-1], [-1], [2], [11, 29], [-1], [-1], [-1], [25, 9], [9], [-1]]
Fitness -  0
Code -  [True, True, False, True, True, True, False, False, False, False, False, False, False, False, True, False, True, True, False, True, False, True, True, True, False, False, True, True, True, False, False, True]
